# Philodoptera Dataset Conversion

Convert cricket tracking data to movement-compatible poses.csv format and then to TrialTree.

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import xarray as xr
from pathlib import Path
from audioio import load_audio

from movement.io import load_poses
from movement.kinematics import compute_velocity, compute_speed, compute_acceleration

from ethograph.utils.io import TrialTree, set_media_attrs, add_changepoints_to_ds
from ethograph.features.audio_features import get_synced_envelope
from ethograph.features.changepoints import find_troughs_binary, find_nearest_turning_points_binary

In [ ]:
data_folder = Path(r"C:\Users\aksel\Documents\Code\EthoGraph\data\Philodoptera")

pos_path = data_folder / "exported_labels.csv"
audio_path = data_folder / "audio.wav"
video_path = data_folder / "video3.mp4"

fps = 240
scorer = "Akseli"

## Step 1: Load and reshape raw tracking data

In [ ]:
df_raw = pd.read_csv(pos_path)
print(f"Raw data shape: {df_raw.shape}")
print(f"Columns: {df_raw.columns.tolist()}")
print(f"\nFirst few rows:")
df_raw.head(10)

In [ ]:
# Data has alternating rows: even indices = left wingtip, odd indices = right wingtip
# Each frame appears twice (once for each keypoint)

x = df_raw["x"].values.copy()
y = df_raw["y"].values.copy()
frames = df_raw["frame"].values

# Replace 0 with NaN (missing tracking)
x[x == 0] = np.nan
y[y == 0] = np.nan

# Split into left and right wingtips
x_left = x[::2]    # even indices
x_right = x[1::2]  # odd indices
y_left = y[::2]
y_right = y[1::2]
frame_nums = frames[::2]  # unique frame numbers

n_frames = len(frame_nums)
print(f"Number of frames: {n_frames}")
print(f"Frame range: {frame_nums[0]} to {frame_nums[-1]}")
print(f"Duration: {n_frames / fps:.2f} seconds")

## Step 2: Create DLC-style DataFrame with multi-level header

In [ ]:
# DeepLabCut CSV format has multi-level columns:
# Level 0: scorer
# Level 1: bodyparts 
# Level 2: coords (x, y, likelihood)

bodyparts = ["LeftWingTip", "RightWingTip"]
coords = ["x", "y", "likelihood"]

# Create multi-index columns
columns = pd.MultiIndex.from_product(
    [[scorer], bodyparts, coords],
    names=["scorer", "bodyparts", "coords"]
)

# Build data array: shape (n_frames, n_bodyparts * 3)
# Order: LeftWingTip_x, LeftWingTip_y, LeftWingTip_likelihood, RightWingTip_x, ...
likelihood = np.ones(n_frames)  # No confidence data, set to 1

data = np.column_stack([
    x_left, y_left, likelihood,
    x_right, y_right, likelihood
])

df_dlc = pd.DataFrame(data, columns=columns)
print(f"DLC DataFrame shape: {df_dlc.shape}")
df_dlc.head()

In [ ]:
# Save as DLC-style CSV
poses_csv_path = data_folder / "poses.csv"
df_dlc.to_csv(poses_csv_path)
print(f"Saved DLC-style CSV to: {poses_csv_path}")

## Step 3: Load poses with movement library

In [ ]:
# Load using movement's from_dlc_style_df (directly from DataFrame)
ds = load_poses.from_dlc_style_df(df_dlc, fps=fps)
print(f"Dataset dimensions: {dict(ds.dims)}")
print(f"\nData variables: {list(ds.data_vars)}")
print(f"\nCoordinates:")
for coord in ds.coords:
    print(f"  {coord}: {ds.coords[coord].values}")

In [ ]:
# Verify the loaded data matches our original
pos = ds.position.values
print(f"Position shape: {pos.shape}")
print(f"  Expected: (time={n_frames}, space=2, keypoints=2, individuals=1)")

# Check first valid frame (around frame 306 based on original notebook)
valid_idx = np.where(~np.isnan(x_left))[0][0]
print(f"\nFirst valid frame index: {valid_idx}")
print(f"Original LeftWingTip x,y: ({x_left[valid_idx]:.2f}, {y_left[valid_idx]:.2f})")
print(f"Loaded position: {ds.position.isel(time=valid_idx, individuals=0).values}")

## Step 4: Compute kinematics

In [ ]:
ds["velocity"] = compute_velocity(ds.position)
ds["speed"] = compute_speed(ds.position)
ds["acceleration"] = compute_acceleration(ds.position)

print("Added kinematics:")
for var in ["velocity", "speed", "acceleration"]:
    print(f"  {var}: {ds[var].dims}")

## Step 5: Add audio envelope

In [ ]:
audio, sr = load_audio(str(audio_path))
print(f"Audio shape: {audio.shape}")
print(f"Sample rate: {sr} Hz")
print(f"Audio duration: {audio.shape[0] / sr:.2f} seconds")
print(f"Video duration: {n_frames / fps:.2f} seconds")

In [ ]:
# Get synced envelope (resampled to video fps)
envelope, _ = get_synced_envelope(str(audio_path), sr, fps)
print(f"Envelope shape: {envelope.shape}")
print(f"Expected frames: {n_frames}")

# Trim or pad envelope to match video frames
if len(envelope) > n_frames:
    envelope = envelope[:n_frames]
    print(f"Trimmed envelope to {n_frames} frames")
elif len(envelope) < n_frames:
    envelope = np.pad(envelope, (0, n_frames - len(envelope)), mode='constant', constant_values=np.nan)
    print(f"Padded envelope to {n_frames} frames")

In [ ]:
# Add envelope to dataset
ds["envelope"] = xr.DataArray(
    envelope,
    dims=["time"],
    coords={"time": ds.coords["time"]}
)
ds["envelope"].attrs["type"] = "features"

# Store audio metadata
ds.attrs["audio_sr"] = sr
ds.attrs["fps"] = fps

## Step 6: Add changepoints

In [ ]:
# Add changepoints on envelope (audio peaks/troughs)
ds = add_changepoints_to_ds(
    ds=ds,
    target_feature="envelope",
    changepoint_name="troughs",
    changepoint_func=find_troughs_binary,
    prominence=0.01,
    distance=5
)

ds = add_changepoints_to_ds(
    ds=ds,
    target_feature="envelope",
    changepoint_name="turning_points",
    changepoint_func=find_nearest_turning_points_binary,
    threshold=0.01,
    max_value=1.0,
    prominence=0.02,
    width=2
)

print("Changepoint variables added:")
for var in ds.data_vars:
    if "changepoints" in str(ds[var].attrs.get("type", "")):
        print(f"  {var}")

## Step 7: Set media attributes and create TrialTree

In [ ]:
# Mark features
for var in ["position", "velocity", "speed", "acceleration", "confidence"]:
    if var in ds.data_vars:
        ds[var].attrs["type"] = "features"

# Set media attributes
ds = set_media_attrs(
    ds,
    cameras=[video_path.name],
    mics=[audio_path.name],
    tracking=[poses_csv_path.name],
    tracking_prefix="dlc"
)

# Add labels array (required for TrialTree)
individuals = ds.coords["individuals"].values
ds["labels"] = xr.DataArray(
    np.zeros((len(ds.coords["time"]), len(individuals))),
    dims=["time", "individuals"],
    coords={"time": ds.coords["time"], "individuals": individuals}
)

# Set trial attribute
ds.attrs["trial"] = "stridulation_1"
ds.attrs["source_software"] = "DeepLabCut"

print("Dataset attributes:")
for k, v in ds.attrs.items():
    print(f"  {k}: {v}")

In [ ]:
# Create TrialTree
dt = TrialTree.from_datasets([ds])
print(dt)

In [ ]:
# Save to NetCDF
output_path = data_folder / "Philodoptera.nc"
dt.to_netcdf(output_path)
print(f"Saved TrialTree to: {output_path}")

## Step 8: Verify and visualize

In [ ]:
# Load back and verify
dt_loaded = TrialTree.load(str(output_path))
print(dt_loaded)
print(f"\nTrials: {dt_loaded.trials}")

In [ ]:
import matplotlib.pyplot as plt

ds_trial = dt_loaded.itrial(0)
time = ds_trial.coords["time"].values

fig, axs = plt.subplots(3, 1, figsize=(14, 8), sharex=True)

# Plot envelope
axs[0].plot(time, ds_trial["envelope"].values, color="green", alpha=0.7)
axs[0].set_ylabel("Audio Envelope")
axs[0].set_title("Philodoptera Stridulation Dataset")

# Plot x positions
pos = ds_trial["position"].sel(space="x", individuals="individual_0").values
for i, kp in enumerate(ds_trial.coords["keypoints"].values):
    axs[1].plot(time, pos[:, i], label=kp)
axs[1].set_ylabel("X Position (px)")
axs[1].legend()

# Plot speed
speed = ds_trial["speed"].sel(individuals="individual_0").values
for i, kp in enumerate(ds_trial.coords["keypoints"].values):
    axs[2].plot(time, speed[:, i], label=kp)
axs[2].set_ylabel("Speed (px/s)")
axs[2].set_xlabel("Time (s)")
axs[2].legend()

plt.tight_layout()
plt.show()